In [ ]:
#| default_exp tui


# TUI - Terminal User Interface

Textual-based terminal user interface components for audio transcription.


In [ ]:
#| export
from time import monotonic
from enum import Enum, auto
import asyncio

from textual import on, events
from textual.app import App, ComposeResult
from textual.reactive import reactive
from textual.containers import Container, Horizontal, Vertical, Grid
from textual.screen import ModalScreen
from textual.message import Message
from textual.widget import Widget
from textual.widgets import Header, Footer, Static, Digits, Button, Label, Log, Select, Rule

from tui_writer.live import LiveTranscriber
from tui_writer.ai import start_session, apply_instruction, reset_session

In [ ]:
#| export

TEXTUAL_CSS = """
    Screen {
        align: center top;
    }
    Header {
        background: $background;
    }

    #top-bar {
        hatch: cross $warning 50%;
        width: 90%;
        min-width: 100;
        align: center middle;
        margin-top: 1;
        height: auto;
    }

    .recording #top-bar {
        hatch: right $error 50%;
    }

    #status {
        hatch: cross $warning 10%;
        width: 50;
        content-align: center middle;
        padding: 1 0;
    }

    #media-controls {
        hatch: cross $warning 10%;
        width: 50;
        align: center middle;
        height: auto;
        padding: 1 0;
    }
    
    #main-textarea {
        hatch: horizontal $boost 80%;
        background: $boost;
        width: 90%;
        min-width: 100;
        padding: 2 3;
    }

    SettingsModal {
        align: center middle;
    }
    #settings {
        border: thick $background 80%;
        background: $surface;
        width: 100;
        padding: 0 2;
        align: center top;
        height: 20;
    }
    
    #modal-header {
        content-align: center middle;
        margin: 1 0;
    }

    Rule {
        margin: 0;
    }

    #selections {
        layout: grid;
        grid-size: 3 3;
        grid-rows: 1fr;
        grid-columns: 1fr;
        grid-gutter: 1;
    }
    .select-text {
        height: 100%;
        content-align: center middle;
    }
    Select {
        column-span: 2;
    }
    #save-container {
        column-span: 3;
        align: right bottom;
    }
"""

In [ ]:
#| export

class SettingsModal(ModalScreen):
    # Available Whisper model options (label, value)
    WHISPER_MODELS = [
        ("Tiny      ⚡  Ultra fast   - Low accuracy", "tiny"),
        ("Base      ⚡  Fast         - Decent accuracy", "base"),
        ("Small     ⚖   Balanced    - Good accuracy", "small"),
        ("Medium    🐢  Slow        - High accuracy",  "medium"),
        ("Large     🐢  Very Slow   - Best accuracy", "large-v3"),
        ("Tiny.en   English-only    - faster", "tiny.en"),
        ("Base.en   English-only    - faster", "base.en"),
        ("Small.en  English-only    - faster", "small.en"),
        ("Medium.en English-only    - faster", "medium.en"),
        ("Large.en  English-only    - faster", "large-v3.en")
    ]
    WHISPER_LANGUAGES = [
        ("English", "en"),
        ("Norwegian", "no"),
        ("Swedish", "sv"),
        ("Danish", "da"),
        ("Finnish", "fi"),
        ("German", "de"),
        ("Dutch", "nl"),
        ("French", "fr"),
        ("Spanish", "es"),
        ("Portuguese", "pt"),
        ("Italian", "it"),
        ("Polish", "pl"),
        ("Czech", "cs"),
        ("Slovak", "sk"),
        ("Hungarian", "hu"),
        ("Greek", "el"),
        ("Turkish", "tr"),
        ("Russian", "ru"),
        ("Ukrainian", "uk"),
        ("Arabic", "ar"),
        ("Hebrew", "he"),
        ("Hindi", "hi"),
        ("Bengali", "bn"),
        ("Urdu", "ur"),
        ("Persian (Farsi)", "fa"),
        ("Thai", "th"),
        ("Vietnamese", "vi"),
        ("Indonesian", "id"),
        ("Malay", "ms"),
        ("Filipino (Tagalog)", "tl"),
        ("Chinese (Mandarin)", "zh"),
        ("Chinese (Cantonese)", "yue"),
        ("Japanese", "ja"),
        ("Korean", "ko"),
        ("Swahili", "sw"),
        ("Afrikaans", "af"),
        ("Romanian", "ro"),
        ("Bulgarian", "bg"),
        ("Serbian", "sr"),
        ("Croatian", "hr"),
        ("Slovenian", "sl"),
        ("Estonian", "et"),
        ("Latvian", "lv"),
        ("Lithuanian", "lt"),
    ]

    def __init__(self, model, language):
        self.model = model
        self.language = language
        super().__init__()

    def compose(self) -> ComposeResult:
        with Container(id="settings"):
            yield Static("SETTINGS", id="modal-header")
            yield Rule(line_style="heavy")
            with Grid(id="selections"):
                yield Static("Transcribing model:", classes="select-text")
                yield Select(
                        options=self.WHISPER_MODELS,
                        allow_blank=False,
                        value=self.model,
                        id="whisper-model"
                )
                yield Static("Language:", classes="select-text")
                yield Select(
                        options=self.WHISPER_LANGUAGES,
                        allow_blank=False,
                        value=self.language,
                        id="whisper-language"
                )
                yield Container(
                    Button.success("Save (s)", id="save"),
                    id="save-container",
                )


    # Whisper selectors
    @on(Select.Changed, "#whisper-model")
    def on_model_changed(self, event: Select.Changed) -> None:
        self.model = str(event.value)

    @on(Select.Changed, "#whisper-language")
    def on_language_changed(self, event: Select.Changed) -> None:
        self.language = str(event.value)

    def on_button_pressed(self) -> None:
        self.dismiss([self.model, self.language])


## TranscriptionTUI

Main terminal user interface application for audio transcription.

The TUI uses Textual's [theme system](https://textual.textualize.io/guide/design/) with built-in color variables like `$primary`, `$error`, `$success`, etc. 

**Available built-in themes:**
- `textual-dark` (default)
- `textual-light`
- `nord`
- `gruvbox`
- `tokyo-night`
- `solarized-light`

To change the theme, modify the `self.theme` assignment in `__init__`.


In [ ]:
#| export
class RecordingState(Enum):
    """High-level states the TUI can be in."""
    IDLE = auto()       # Not recording
    RECORDING = auto()  # Live transcription is running
    #PAUSED = auto()     # Recording stopped but session kept
    EDIT = auto()       # Voice-based editing mode

class TranscriptionTUI(App):
    """
    Main Textual app for live transcription and AI-assisted editing.

    - Handles recording, pausing, resuming, and editing voice input in real time.
    - Integrates Whisper for transcription and an AI model for text editing.
    - Uses reactive state management (RecordingState) to update the UI automatically.

    The design principle: 
    - `_start()` and `_stop()` handle *recording logic only*.
    - `watch_state()` updates the UI reactively when state changes.
    - Event handlers (buttons, keybindings) change state and call those methods.
    """

    TITLE = "TUI Writer"
    SUB_TITLE = "Design Template"
    CSS = TEXTUAL_CSS
    AUTO_FOCUS = False



    # Reactive state triggers watch_state() automatically when changed
    state = reactive(RecordingState.IDLE)

    # Keyboard bindings (TUI shortcuts)
    BINDINGS = [
        ("q", "quit", "Quit"),
        ("space", "toggle_recording", "Start/Stop"),
        ("s", "settings_modal", "Settings"),
    ]

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # Async background task for transcription loop
        self._task: asyncio.Task | None = None
        # The active LiveTranscriber instance
        self._transcriber: LiveTranscriber | None = None
        # Full transcript text accumulated so far
        self.all_chunks: str = ""
        # Default Whisper model name
        self.whisper_model = "base"
        # Default whisper language
        self.language = "en"

    def compose(self) -> ComposeResult:
        """Construct and layout all UI widgets."""
        self.start_btn = Button.error("● REC <space>", id="start")
        self.stop_btn = Button("◼ STOP <space>", id="stop")

        yield Header(show_clock=True)

        # Top section (status + info + current operations)
        with Container(id="top-bar"):
            yield Static("○ STANDBY", id="status")
            with Horizontal(id="media-controls"):
                yield self.start_btn
                yield self.stop_btn
        yield Log(id="main-textarea", auto_scroll=True)

       


        # Bottom section (main transcript log)
        
        yield Footer()

    # Action methods triggered by key bindings, refers to button-methods
    async def action_toggle_recording(self) -> None:
        """Spacebar: start if idle, stop if currently recording."""
        if self.state is RecordingState.IDLE:
            await self._start()
            self.state = RecordingState.RECORDING
        elif self.state is RecordingState.RECORDING:
            await self._stop()
            self.state = RecordingState.IDLE
        
    def action_settings_modal(self) -> None:
        self.push_screen(SettingsModal(self.whisper_model, self.language), self.apply_settings)

    def apply_settings(self, ting):
        self.whisper_model = ting[0]
        self.language = ting[1]

    # Button event handlers (mapped in the UI)
    @on(Button.Pressed, "#start")
    async def _on_start(self, _):
        """Start button pressed."""
        if self.state is RecordingState.IDLE:
            await self._start()
            self.state = RecordingState.RECORDING

    @on(Button.Pressed, "#stop")
    async def _on_stop(self, _):
        """Stop button pressed."""
        if self.state is RecordingState.RECORDING:
            await self._stop()
            self.state = RecordingState.IDLE

    async def on_key(self, event: events.Key) -> None:
        """Press 'e' to toggle live edit mode."""
        if event.key != "e":
            return
        if self.state is not RecordingState.EDIT:
            await self._start_edit()
            self.state = RecordingState.EDIT 
        else:
            # Leaving edit mode
            self.transcript_block.loading = True
            self.main_textarea.loading = True
            await self._stop_edit()
            self.transcript_block.loading = False
            self.main_textarea.loading = False
            self.plan = None
            self.edit_instructions = ""
            self.state = RecordingState.IDLE

    # Live edit mode helpers
    async def _start_edit(self) -> None:
        """Start edit session and initialize and start transcriber."""
        self.current_transcript = start_session(self.all_chunks)
        self._transcriber = LiveTranscriber(
            model_id=self.whisper_model,
            language=self.language,
            on_transcript=self.on_edit_transcript,
            vad_threshold=0.5,
            min_speech_duration_ms=250,
            min_silence_duration_ms=500,
        )
        self._task = asyncio.create_task(self._transcriber.start())

    async def _stop_edit(self) -> None:
        """Stop edit mode and apply queued voice edit instructions."""
        try:
            if self._transcriber:
                self._transcriber.stop()
            if self._task:
                await self._task
            if self._transcriber:
                await self._transcriber.flush()
        finally:
            self._transcriber = None
            self._task = None
            # Apply accumulated voice instructions to current transcript
            self.current_transcript, self.plan = apply_instruction(self.edit_instructions)
            reset_session()
            self.all_chunks = self.current_transcript
            # Show resulting plan and updated transcript in UI
            ops_text = f"Plan: {self.plan.model_dump_json(indent=2)}"
            self.transcript_block.update(ops_text)
            self.main_textarea.clear()
            self.main_textarea.write_lines(self.all_chunks.splitlines(True))

    # Transcription lifecycle
    async def _start(self) -> None:
        """Initialize and start live transcription."""
        self._transcriber = LiveTranscriber(
            model_id=self.whisper_model, language=self.language,
            on_transcript=self.on_transcript_chunk,
            vad_threshold=0.5, min_speech_duration_ms=250, min_silence_duration_ms=500,
        )
        self._task = asyncio.create_task(self._transcriber.start())
        self.start_btn.disabled = True
        self.stop_btn.disabled = False

    async def _stop(self) -> None:
        """Stop active transcriber and cleanup task."""
        try:
            if self._transcriber:
                self._transcriber.stop()
            if self._task:
                await self._task
            if self._transcriber:
                await self._transcriber.flush()
        finally:
            self._transcriber = None
            self._task = None
            self.start_btn.disabled = False
            self.stop_btn.disabled = True

    # Callback handlers
    def on_transcript_chunk(self, text: str) -> None:
        """Called whenever the transcriber produces a new text chunk."""
        text = (text or "").strip()
        if not text or self.state is RecordingState.IDLE: # possible dont need to check self.state
            return
        self.all_chunks += text + "\n"
        self.main_textarea.write_line(text)
    
    def on_edit_transcript(self, instruction: str) -> None:
        """Collects spoken edit instructions during edit mode."""
        instruction = (instruction or "").strip()
        if not instruction:
            return
        prev = getattr(self, "edit_instructions", "")
        self.edit_instructions = (prev + ("\n" if prev else "") + instruction)

    # UI state watcher (reactive)
    def watch_state(self, new_state: RecordingState) -> None:
        """Reactively update UI when recording state changes."""

        match new_state:
            case RecordingState.IDLE:
                self.status_text.update("○ STANDBY")
            case RecordingState.RECORDING:
                self.status_text.update("● RECORDING...")
            case RecordingState.EDIT:
                self.status_text.update("● LIVE EDITING...")

    # Mount / Unmount lifecycle
    def on_mount(self) -> None:
        """Initialize widget references and set titles."""
        self.status_text: Static = self.query_one("#status", Static)
        self.main_textarea: Log = self.query_one("#main-textarea", Log)
        self.stop_btn: Button = self.query_one("#stop", Button)
        self.stop_btn.disabled = True
        self.theme = "textual-dark"
        self.main_textarea.write_line("This text is just for testing.\nIt's already loaded once i start the tui.\nA new line is started once you start talking again.")

    async def on_unmount(self) -> None:
        """Cleanly stop background tasks when app closes."""
        await self._stop()